In [15]:
from fishsense_api_sdk.client import Client
from label_studio_sdk.client import LabelStudio
from label_studio_sdk.core import ApiError
from tqdm.notebook import tqdm
from pathlib import Path
from tqdm.asyncio import tqdm_asyncio
from fishsense_api_sdk.models.dive_slate_label import DiveSlateLabel
import json

In [16]:
URL = "http://localhost:8000"

In [17]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [18]:
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [19]:
async with Client(URL) as fs:
    dives = await fs.dives.get_canonical()

len(dives), dives[0]

(272,
 Dive(id=1, name='080123_FSL-01 Photos', path='2023-09-07 REEF Data Dump/080123_FSL-01 Photos', dive_datetime=datetime.datetime(2023, 8, 1, 12, 46, 27, tzinfo=TzInfo(0)), priority=<Priority.LOW: 'LOW'>, flip_dive_slate=True, camera_id=1, dive_slate_id=1))

In [20]:
async with Client(URL) as fs:
    slate_labels = await tqdm_asyncio.gather(*[fs.labels.get_dive_slate_labels(dive.id) for dive in dives])
    slate_labels = [label for sublist in slate_labels for label in sublist]

len(slate_labels), slate_labels[0] 

100%|██████████| 272/272 [00:06<00:00, 43.14it/s]


(95,
 DiveSlateLabel(id=1, label_studio_task_id=223832, label_studio_project_id=66, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/2863b55ffedf2bbb1e94f8042fd8ad60', upside_down=None, reference_points=None, slate_rectangle=None, skipped_points=None, updated_at=None, completed=False, label_studio_json={}, image_id=78686, user_id=None))

In [21]:
label_studio_project_ids = list({label.label_studio_project_id for label in slate_labels if label.label_studio_project_id is not None})

len(label_studio_project_ids), label_studio_project_ids

(1, [66])

In [22]:
tasks = []

for project_id in tqdm(label_studio_project_ids):
    try:
        project = ls.projects.get(project_id)
    except ApiError as e:
        print(f"Error fetching project {project_id}: {e}")
        continue

    tasks += list(ls.tasks.list(project=project_id))

len(tasks), tasks[0]

  0%|          | 0/1 [00:00<?, ?it/s]

(95,
 LseTask(agreement=None, agreement_selected=None, annotations=[{'id': 130421, 'result': [{'id': '5dkO2_2wp4', 'type': 'choices', 'value': {'choices': ['Slate upside down']}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'upside_down'}, {'id': 'tKw3uftfpN', 'type': 'keypointlabels', 'value': {'x': 78.6283259928739, 'y': 31.806675638557287, 'width': 0.1061148693460671, 'keypointlabels': ['Reference Point']}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'reference_points', 'image_rotation': 0, 'original_width': 7917, 'original_height': 3016}, {'id': 'oo-hdGoYQ8', 'type': 'keypointlabels', 'value': {'x': 76.7258232131775, 'y': 31.806675638557287, 'width': 0.1061148693460671, 'keypointlabels': ['Reference Point']}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'reference_points', 'image_rotation': 0, 'original_width': 7917, 'original_height': 3016}, {'id': 'AR0C2PrSp3', 'type': 'keypointlabels', 'value': {'x': 74.56965339618827, 'y': 31.91765496737291, 'width': 0.

In [23]:
label_studio_task_to_laser_label_map = {
    label.label_studio_task_id: label
    for label in slate_labels
}

len(label_studio_task_to_laser_label_map)

95

In [58]:
coroutines = []
async with Client(URL) as fs:
    for task in tqdm(tasks):
        dive_slate_label: DiveSlateLabel = label_studio_task_to_laser_label_map.get(task.id)

        if dive_slate_label is None:
            continue

        dive_slate_label.label_studio_json = json.loads(task.json())
        dive_slate_label.completed = task.is_labeled

        if len(task.annotations) > 0:
            email = task.annotations[0]["created_username"].split(",")[0].split(' ')[-1].strip()
            tqdm.write(f"Processing task {task.id} for user {email}")
            user = await fs.users.get(email=email)
            dive_slate_label.user_id = user.id

            upside_down = [r for r in task.annotations[0]["result"] if r["from_name"] == "upside_down"]
            upside_down = upside_down[0]["value"]["choices"][0] == "Slate upside down" if len(upside_down) > 0 else None
            dive_slate_label.upside_down = upside_down

            reference_points = [(r["value"]["x"] / 100 * r["original_width"], r["value"]["y"] / 100 * r["original_height"]) for r in task.annotations[0]["result"] if r["from_name"] == "reference_points"]
            dive_slate_label.reference_points = reference_points

            slate_rectangle = [r for r in task.annotations[0]["result"] if r["from_name"] == "slate"]
            slate_rectangle = slate_rectangle[0] if len(slate_rectangle) > 0 else None
            slate_rectangle = [
                (slate_rectangle["value"]["x"] / 100 * slate_rectangle["original_width"], slate_rectangle["value"]["y"] / 100 * slate_rectangle["original_height"]),
                ((slate_rectangle["value"]["x"] + slate_rectangle["value"]["width"]) / 100 * slate_rectangle["original_width"], (slate_rectangle["value"]["y"] + slate_rectangle["value"]["height"]) / 100 * slate_rectangle["original_height"]),
            ] if slate_rectangle is not None else None
            dive_slate_label.slate_rectangle = slate_rectangle
            
            skipped_points = [r for r in task.annotations[0]["result"] if r["from_name"] == "skipped_points"]
            skipped_points = skipped_points[0] if len(skipped_points) > 0 else None
            skipped_points = [int(point) - 1 for point in skipped_points["value"]["text"]] if skipped_points is not None else None
            dive_slate_label.skipped_points = skipped_points        

        coroutines.append(fs.labels.put_dive_slate_label(dive_slate_label.image_id, dive_slate_label))

    await tqdm_asyncio.gather(*coroutines)

  0%|          | 0/95 [00:00<?, ?it/s]

Processing task 223832 for user ccrutchf@ucsd.edu
Processing task 223833 for user ccrutchf@ucsd.edu


/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-packages/label_studio_sdk/core/pydantic_utilities.py:88: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='annotations', input_value=[{'id': 130422, 'result':...last_created_by': None}], input_type=list])
  serialized = handler(self)
/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-packages/label_studio_sdk/core/pydantic_utilities.py:88: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='annotations', input_value=[{'id': 130423, 'result':...last_created_by': None}], input_type=list])
  serialized = handler(self)


Processing task 223834 for user ccrutchf@ucsd.edu
Processing task 223835 for user ccrutchf@ucsd.edu


/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-packages/label_studio_sdk/core/pydantic_utilities.py:88: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='annotations', input_value=[{'id': 130424, 'result':...last_created_by': None}], input_type=list])
  serialized = handler(self)
/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-packages/label_studio_sdk/core/pydantic_utilities.py:88: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='annotations', input_value=[{'id': 130425, 'result':...last_created_by': None}], input_type=list])
  serialized = handler(self)
/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-package

...

/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-packages/label_studio_sdk/core/pydantic_utilities.py:88: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='annotations', input_value=[{'id': 130547, 'result':...last_created_by': None}], input_type=list])
  serialized = handler(self)
/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-packages/label_studio_sdk/core/pydantic_utilities.py:88: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='annotations', input_value=[{'id': 130550, 'result':...last_created_by': None}], input_type=list])
  serialized = handler(self)


Processing task 223923 for user ftmchenry@live.com
Processing task 223924 for user ccrutchf@ucsd.edu


/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-packages/label_studio_sdk/core/pydantic_utilities.py:88: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='annotations', input_value=[{'id': 130551, 'result':...last_created_by': None}], input_type=list])
  serialized = handler(self)
/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-packages/label_studio_sdk/core/pydantic_utilities.py:88: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='annotations', input_value=[{'id': 130553, 'result':...last_created_by': None}], input_type=list])
  serialized = handler(self)


Processing task 223925 for user ftmchenry@live.com
Processing task 223926 for user ccrutchf@ucsd.edu


/home/chris/Repos/school/e4e/fishsense/fishsense-data-processing-workflow-worker/.venv/lib/python3.13/site-packages/label_studio_sdk/core/pydantic_utilities.py:88: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='annotations', input_value=[{'id': 130555, 'result':...last_created_by': None}], input_type=list])
  serialized = handler(self)
100%|██████████| 95/95 [00:02<00:00, 34.73it/s]


In [ ]:
skipped_points = [r for r in task.annotations[0]["result"] if r["from_name"] == "skipped_points"]
skipped_points = skipped_points[0] if len(skipped_points) > 0 else None
skipped_points = [int(point) - 1 for point in skipped_points["value"]["text"]]

skipped_points

[4]